In [1]:
import numpy as np
import pandas as pd
#import torch
from importlib import reload
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score
from sklearn.impute import SimpleImputer
import midi.utils; reload(midi.utils)
from midi.utils import benchmark_with_multiple_models

In [2]:
rna_seq = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_rnaseq_genes_145subjects_17072genes.npy")
acgh = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_1mb_aCGH.npy")
clinical = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_clinical_data_for_fit.npy")

stages = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_stages.npy")
stages = np.array(stages).flatten().astype(str)
#stages

y = np.load("/home/vero/projects/camda/NB/data_cleaned/dataset_outputs.npy")
mergeable = pd.read_csv("/home/vero/projects/camda/NB/data_cleaned/clinical_data_for_fit.csv", index_col=0)
clinical_HR = pd.read_csv("/home/vero/projects/camda/NB/data_cleaned/info_patients_clinical.csv", 
                          index_col=0)['high_risk2']
clinical_HR.sort_index()
HR = clinical_HR.loc[mergeable.index].values
HR[np.where(HR=='HR')] = '1'
HR = HR.astype(int)

In [6]:
y_new = y[:,1]
y_new[np.where(y_new==0)] = -1

# Classification Ensemble elastic net

In [119]:
import midi.utils; reload(midi.utils)
import midi.intermediate.ensemble.classification; reload(midi.intermediate.ensemble.classification)

from midi.intermediate.ensemble.classification import EnsembleClassifier

In [120]:
#model = EnsembleClassifier(estimator_params=[(1,0.5), (1,1), (0.5, 0.5), (2, 0.5)], verbose=1, tol=1e-5)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
import midi.intermediate.ensemble.cross_validation; reload(midi.intermediate.ensemble.cross_validation)
from midi.intermediate.ensemble.cross_validation import EnsembleCV;
tr_scores = []
ts_scores = []
parameters = []
val_scores = []
sss = StratifiedShuffleSplit(n_splits=100)
for train, test in sss.split(rna_seq, y):
    sss_ = StratifiedKFold(n_splits=3)
    x_tr = rna_seq[train,:]
    y_tr = y_new[train]
    x_ts = rna_seq[test, :]
    y_ts = y_new[test]
    model =  EnsembleClassifier(estimator_params=[(1,0.5), (1,1), (0.5, 0.5), (2, 0.5)],
                                verbose=0, tol=1e-5)
    cv = EnsembleRandomSearchCV(model, parameters, cv=sss)
    cv.fit(x_tr, y_tr)
    parameters.append(cv.best_params_)
    val_scores.append(cv.best_score_)
    tr_scores.append(balanced_accuracy_score(y_tr, cv.predict(x_tr)))
    ts_scores.append(balanced_accuracy_score(y_ts, cv.predict(x_ts)))

In [123]:
model.coef_, model.W_

(array([0.85651755, 0.68393795, 1.41731967]),
 [array([-1.50368692e-05, -3.35109977e-06, -1.39017322e-05, ...,
         -5.47501349e-07, -2.01964426e-05,  8.76785874e-06]),
  array([ 8.99965534e-06,  1.30543196e-05, -2.42069897e-05, ...,
          2.23918678e-05,  4.86524277e-06,  1.14257088e-05]),
  array([1.65600381e-04, 1.53295199e-05])])

In [124]:
from sklearn.metrics import balanced_accuracy_score
R = []
for w, X in zip(model.W_,[rna_tr, acgh_tr, cl_tr] ):
    R.append(X.dot(w))
    #print(balanced_accuracy_score(y_new[train], np.sign(X.dot(w))))
R = np.array(R)
#print(R)
res = np.zeros_like(R[0])
for i in range(len(model.coef_)):
    res += model.coef_[i]*R[i]
print(balanced_accuracy_score(y_new[train], np.sign(res)))
#np.sign(res)

0.5977328431372549


In [125]:
#model.predict([rna_tr])
model.predict([rna_tr, acgh_tr, cl_tr])

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,
       -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,
        1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.])

In [126]:
y_new[train]

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        1,  1,  1, -1, -1, -1,  1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1,
       -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1,  1,  1, -1,  1, -1,  1, -1, -1, -1, -1,
        1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
        1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1, -1,  1, -1, -1,
        1, -1, -1,  1, -1, -1, -1,  1,  1,  1, -1])

In [127]:
#model.score([rna_tr], y_new[train])
model.score([rna_tr, acgh_tr, cl_tr], y_new[train])

0.5977328431372549

In [ ]:
rna_ts = rna_imp.fit_transform(rna_seq[test,:])
acgh_ts = acgh[test,:]
acgh_ts[np.where(np.isnan(acgh_ts))] = 0
cl_ts = clinical[test,:]
cl_ts[np.where(np.isnan(cl_ts))] = 0
y_ts = y[test,1]

In [ ]:
model.score([rna_ts, acgh_ts, cl_ts], y_ts)

In [ ]:
from sklearn.linear_model import RidgeClassifier
m = RidgeClassifier()
m.fit(rna_tr, y_new[train])

In [ ]:
m.coef_